# Convolutional Neural Networks: Application

이 튜토리얼은 MNIST 숫자를 분류하기 위해 간단한 합성곱 신경망(Convolutional Neural Network, CNN)을 훈련합니다. 간단한 이 네트워크는 MNIST 테스트 세트에서 99% 정확도를 달성할 것입니다. 이 튜토리얼은 케라스 Sequential API를 사용하기 때문에 몇 줄의 코드만으로 모델을 만들고 훈련할 수 있습니다.

### 텐서플로 임포트하기

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

In [2]:
### MNIST 데이터셋 다운로드하고 준비하기

(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# 픽셀 값을 0~1 사이로 정규화합니다.
train_images, test_images = train_images / 255.0, test_images / 255.0

print(train_images.shape, train_labels.shape)

(60000, 28, 28, 1) (60000,)


#### 합성곱 층 만들기
아래 6줄의 코드에서 Conv2D와 MaxPooling2D 층을 쌓는 일반적인 패턴으로 합성곱 층을 정의합니다.

CNN은 배치(batch) 크기를 제외하고 (이미지 높이, 이미지 너비, 컬러 채널) 크기의 텐서(tensor)를 입력으로 받습니다. MNIST 데이터는 (흑백 이미지이기 때문에) 컬러 채널(channel)이 하나지만 컬러 이미지는 (R,G,B) 세 개의 채널을 가집니다. 이 예에서는 MNIST 이미지 포맷인 (28, 28, 1) 크기의 입력을 처리하는 CNN을 정의하겠습니다. 이 값을 첫 번째 층의 input_shape 매개변수로 전달합니다.

In [3]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

지금까지 모델의 구조를 출력해 보죠.

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


위에서 Conv2D와 MaxPooling2D 층의 출력은 (높이, 너비, 채널) 크기의 3D 텐서입니다. 높이와 너비 차원은 네트워크가 깊어질수록 감소하는 경향을 가집니다. Conv2D 층에서 출력 채널의 수는 첫 번째 매개변수에 의해 결정됩니다(예를 들면, 32 또는 64). 일반적으로 높이와 너비가 줄어듦에 따라 (계산 비용 측면에서) Conv2D 층의 출력 채널을 늘릴 수 있습니다.

#### 마지막에 Dense 층 추가하기
모델을 완성하려면 마지막 합성곱 층의 출력 텐서(크기 (4, 4, 64))를 하나 이상의 Dense 층에 주입하여 분류를 수행합니다. Dense 층은 벡터(1D)를 입력으로 받는데 현재 출력은 3D 텐서입니다. 먼저 3D 출력을 1D로 펼치겠습니다. 그다음 하나 이상의 Dense 층을 그 위에 추가하겠습니다. MNIST 데이터는 10개의 클래스가 있으므로 마지막에 Dense 층에 10개의 출력과 소프트맥스 활성화 함수를 사용합니다.

In [5]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

최종 모델의 구조를 확인해 보죠.

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

여기에서 볼 수 있듯이 두 개의 Dense 층을 통과하기 전에 (4, 4, 64) 출력을 (1024) 크기의 벡터로 펼쳤습니다.

#### 모델 컴파일과 훈련하기

In [7]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 42s 22ms/step - loss: 0.1539 - accuracy: 0.95270s - loss: 0.1541 - accuracy: 0.
Epoch 2/5
1875/1875 [==============================] - 42s 22ms/step - loss: 0.0460 - accuracy: 0.9855
Epoch 3/5
1875/1875 [==============================] - 40s 22ms/step - loss: 0.0323 - accuracy: 0.9900
Epoch 4/5
1875/1875 [==============================] - 43s 23ms/step - loss: 0.0250 - accuracy: 0.9920
Epoch 5/5
1875/1875 [==============================] - 43s 23ms/step - loss: 0.0203 - accuracy: 0.99351s - - ETA


#### 모델 평가

In [8]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(test_acc)

313/313 - 2s - loss: 0.0493 - accuracy: 0.9855
0.9854999780654907


결과에서 보듯이 간단한 CNN 모델이 99%의 테스트 정확도를 달성합니다. 몇 라인의 코드치고 나쁘지 않네요! (케라스의 서브클래싱 API와 GradientTape를 사용하여) CNN을 만드는 또 다른 방법은 [여기](https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tutorials/quickstart/advanced.ipynb)를 참고하세요.

## 2.0 - 수신호 인식하기 예제

다음 셀을 실행하여 사용할 "SIGNS"데이터 세트를 로드합니다.

In [9]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

# Loading the data (signs)
train_dataset = h5py.File('train_signs.h5', "r")
print(train_dataset.keys())
X_train_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
Y_train= np.array(train_dataset["train_set_y"][:]) # your train set labels

test_dataset = h5py.File('test_signs.h5', "r")
print(test_dataset.keys())
X_test_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
Y_test = np.array(test_dataset["test_set_y"][:]) # your test set labels

<KeysViewHDF5 ['list_classes', 'train_set_x', 'train_set_y']>
<KeysViewHDF5 ['list_classes', 'train_set_x', 'train_set_y']>


KeyError: "Unable to open object (object 'test_set_x' doesn't exist)"

참고로 SIGNS 데이터 세트는 0에서 5까지의 숫자를 나타내는 6 개의 기호 모음입니다.

<img src="images/SIGNS.png" style="width:800px;height:300px;">

다음 셀은 데이터 세트에서 라벨이 지정된 이미지의 예를 보여줍니다. 아래의 'index'값을 자유롭게 변경하고 다시 실행하여 다른 예를 확인하세요. 

In [ ]:
# Example of a picture
index = 2
plt.imshow(X_train_orig[index])
print ("y = " + str(Y_train[index]))

시작하려면 데이터의 형태를 살펴 보겠습니다.

In [ ]:
X_train = X_train_orig/255.
X_test = X_test_orig/255.
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

**(1) 연습문제**

CONV2D with RELU-> MAXPOOL-> CONV2D with RELU-> MAXPOOL-> FLATTEN -> FULLYCONNECTED -> FULLYCONNECTED 모델을 구현하십시오. 
자세하게는 모든 단계에 다음 매개 변수를 사용합니다.

* Conv2D : 64 개 필터, 5x5 필터, 스트라이드 1, activation은 ReLU
* Max Pool : 2x2 필터 크기 및 2x2 스트라이드를 사용합니다.
* Conv2D : 128 개 필터, 3x3 필터, 스트라이드 1, activation은 ReLU
* Max Pool : 2x2 필터 크기와 2x2 스트라이드를 사용합니다.
* Flatten : 이전 출력을 평평하게 합니다.
* FULLYCONNECTED (FC) 층 : 128 유닛, activation은 ReLU, 완전 연결 계층을 적용합니다. 
* FULLYCONNECTED (FC) 층 : 6 유닛, activation은 softmax, 완전 연결 계층을 적용합니다. 

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
## ======= Build Model (~8 lines)









## ========================
model.summary()

**(2) 연습문제**

다음과 같이 모델 학습을 시행 하시오.

* adam optimizer 사용
* sparse_categorical_crossentropy 로 loss 함수 사용
* epoch 10회 사용

In [ ]:
## ======= Train Model (~2 lines)



## =====================

**(4) 연습문제**

시험자료 X_test 와 Y_test로 학습된 모델을 평가하시오.

In [ ]:
## ========= Test Model (~ 1 line)

## =======================
print(test_acc)

In [ ]:
# Example of a picture
index = 12
plt.imshow(X_test_orig[index])
print ("y = " + str(Y_test[index]))